In [ ]:
corpus = 'previews' #'recommended'
path_raw_fiction_previews = '/content/gdrive/MyDrive/fiction_previews/raw_fiction_previews/'

#Import data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

In [ ]:
import pandas as pd
import os, zipfile, pickle
import numpy as np

if corpus == 'recommended':
  
  with zipfile.ZipFile("/content/gdrive/MyDrive/NSU Workshop'21 - Readability/Корпусы/fiction_recommended/fragments.zip", 'r') as zip_ref:
      zip_ref.extractall("/content/1")

  ###################### train
  files = os.listdir("/content/1/fragments/train")
  path = "/content/gdrive/MyDrive/NSU Workshop'21 - Readability/Корпусы/fiction_recommended/ratings_train.csv"
  df = pd.read_csv(path, header = None)

  train_labels = []
  for f in files:
    train_labels.append(df.loc[df[0] == f][1].values[0])

  path = "/content/1/fragments/train/"

  train_texts = []

  for i, f in enumerate(files):
    with open(path + f) as f1:
        content = f1.read()
    train_texts.append(content)

  ###################### test
  files = os.listdir("/content/1/fragments/test")
  path = "/content/gdrive/MyDrive/NSU Workshop'21 - Readability/Корпусы/fiction_recommended/ratings_test.csv"
  df = pd.read_csv(path, header = None)

  test_labels = []
  for f in files:
    test_labels.append(df.loc[df[0] == f][1].values[0])

  path = "/content/1/fragments/test/"

  test_texts = []

  for i, f in enumerate(files):
    with open(path + f) as f1:
        content = f1.read()
    test_texts.append(content)
    
elif corpus == 'previews':

  ###################### train
  files = os.listdir("/content/gdrive/MyDrive/NSU Workshop'21 - Readability/Корпусы/fiction_previews/train")

  train_labels = []

  for f in files:
    if f.find('adult')>=0:
      train_labels.append(1)
    else:
      train_labels.append(0)

  with open(path_raw_fiction_previews + 'train_fiction_previews.pickle', 'rb') as f:
     train_texts = pickle.load(f)
  with open(path_raw_fiction_previews + 'train_fiction_previews_labels.pickle', 'rb') as f:
      train_labels = pickle.load(f)

  ###################### test
  files = os.listdir("/content/gdrive/MyDrive/NSU Workshop'21 - Readability/Корпусы/fiction_previews/test")

  test_labels = []

  for f in files:
    if f.find('adult')>=0:
      test_labels.append(1)
    else:
      test_labels.append(0)

  with open(path_raw_fiction_previews + 'test_fiction_previews.pickle', 'rb') as f:
      test_texts = pickle.load(f)
  with open(path_raw_fiction_previews + 'test_fiction_previews_labels.pickle', 'rb') as f:
      test_labels = pickle.load(f)

#RoBERTa

In [ ]:
!pip install tensorboardX
!pip install simpletransformers

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.metrics import f1_score, recall_score, precision_score

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_data = pd.DataFrame({'text':train_texts, 'labels':train_labels}).sample(frac=1)
test_data = pd.DataFrame({'text':test_texts, 'labels':test_labels})

model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir=True,\
                                output_dir = '/content/gdrive/MyDrive/output', save_steps = 3000, save_model_every_epoch = True,\
                                #train_batch_size = 1, eval_batch_size = 1,\
                                    no_save = False, max_seq_length=512, n_gpu = 1)
model = ClassificationModel('roberta', 'sberbank-ai/ruRoberta-large', args=model_args, num_labels=len(set(train_labels)), use_cuda = True) 

model.train_model(train_data)
result, model_outputs, wrong_predictions = model.eval_model(test_data)

true_res = test_data.labels.values
print(f1_score(model_outputs.argmax(axis = 1), true_res, average = 'weighted'), \
      precision_score(model_outputs.argmax(axis = 1), true_res, average = 'weighted'), \
      recall_score(model_outputs.argmax(axis = 1), true_res, average = 'weighted'))